In [1]:
#|hide
#|eval: false
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
#|hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [11]:
# Python native modules
import os
from copy import deepcopy
# Third party libs
from fastcore.all import *
import numpy as np
# Local modules
from fastai.torch_basics import *

## MultiProcessing Notes

In [10]:
import torchdata.datapipes as dp

In [11]:
dp.iter.IterDataPipe??

Init signature: dp.iter.IterDataPipe(*args, **kwds)
Source:        
class IterDataPipe(IterableDataset[T_co], metaclass=_IterDataPipeMeta):
    r"""
    Iterable-style DataPipe.

    All DataPipes that represent an iterable of data samples should subclass this.
    This style of DataPipes is particularly useful when data come from a stream, or
    when the number of samples is too large to fit them all in memory. ``IterDataPipe`` is lazily initialized and its
    elements are computed only when ``next()`` is called on the iterator of an ``IterDataPipe``.

    All subclasses should overwrite :meth:`__iter__`, which would return an
    iterator of samples in this DataPipe. Calling ``__iter__`` of an ``IterDataPipe`` automatically invokes its
    method ``reset()``, which by default performs no operation. When writing a custom ``IterDataPipe``, users should
    override ``reset()`` if necessary. The common usages include resetting buffers, pointers,
    and various state variables within 

In [18]:
dp.iter.IterDataPipe??

Init signature: dp.iter.IterDataPipe(*args, **kwds)
Source:        
class IterDataPipe(IterableDataset[T_co], metaclass=_IterDataPipeMeta):
    r"""
    Iterable-style DataPipe.

    All DataPipes that represent an iterable of data samples should subclass this.
    This style of DataPipes is particularly useful when data come from a stream, or
    when the number of samples is too large to fit them all in memory. ``IterDataPipe`` is lazily initialized and its
    elements are computed only when ``next()`` is called on the iterator of an ``IterDataPipe``.

    All subclasses should overwrite :meth:`__iter__`, which would return an
    iterator of samples in this DataPipe. Calling ``__iter__`` of an ``IterDataPipe`` automatically invokes its
    method ``reset()``, which by default performs no operation. When writing a custom ``IterDataPipe``, users should
    override ``reset()`` if necessary. The common usages include resetting buffers, pointers,
    and various state variables within 

In [13]:
%%writefile external_run_scripts/notes_multi_proc_82.py
import torchdata.datapipes as dp

class AddABunch1(dp.iter.IterDataPipe):
    def __init__(self,q):
        self.q = [q]

    def __iter__(self):
        for o in range(10): 
            self.q[0].put(o)
            yield o
            
class AddABunch2(dp.iter.IterDataPipe):
    def __init__(self,q):
        self.q = q

    def __iter__(self):
        for o in range(10): 
            self.q.put(o)
            yield o

if __name__=='__main__':
    from torch.multiprocessing import Pool, Process, set_start_method,Queue
    
    try:
        set_start_method('spawn')
    except RuntimeError:
        pass
    
    from torch.utils.data.dataloader_experimental import DataLoader2

    q = Queue()
    dl = DataLoader2(AddABunch2(q),num_workers=1)
    list(dl)
    
    while not q.empty():
        print(q.get())

Overwriting external_run_scripts/notes_multi_proc_82.py


In [5]:
#|hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev import nbdev_export
    nbdev_export()